In [197]:
bm25_all_file_path = 'results/subtask_2a_bm25_english_all_qrels-dev.tsv'
bm25_vclaim_file_path = 'results/subtask_2a_bm25_english_vclaim_qrels-dev.tsv'
bm25_title_file_path = 'results/subtask_2a_bm25_english_title_qrels-dev.tsv'
sbert_file_path = 'results/subtask_2a_sbert_english_qrels-dev.tsv'
N = 20
cnt_vclaims = 13824
cnt_iclaims = 199

In [198]:
import pandas as pd
import csv
import numpy as np

In [199]:
bm25_all_results = pd.read_csv(bm25_all_file_path, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
bm25_vclaim_results = pd.read_csv(bm25_vclaim_file_path, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
bm25_title_results = pd.read_csv(bm25_title_file_path, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])

In [200]:
sbert_results = pd.read_csv(sbert_file_path, sep = '\t', names =['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])

In [201]:
def get_rr_score(results, iclaim, vclaim, index):
    curr_index = index
    while curr_index < len(results):
        if(iclaim == results.loc[curr_index, 'iclaim_id'] and vclaim == results.loc[curr_index, 'vclaim_id']):
            return (results.loc[curr_index, 'score'], 1/ results.loc[curr_index,'rank'])
        curr_index = curr_index + 1

In [202]:
bm25_all_results.head()

iclaim_id  Q0                                   vclaim_id  rank  \
0  tweet-sno-999  Q0  vclaim-sno-single-mother-birth-certificate     1   
1  tweet-sno-999  Q0          vclaim-sno-james-maxwell-trump-son     2   
2  tweet-sno-999  Q0    vclaim-sno-fairfax-county-racism-posting     3   
3  tweet-sno-999  Q0      vclaim-sno-these-things-i-wish-for-you     4   
4  tweet-sno-999  Q0                   vclaim-sno-kenyan-mistake     5   

       score  method  
0  24.390002  elasic  
1  18.510709  elasic  
2  14.801876  elasic  
3  14.652757  elasic  
4  14.346759  elasic

In [203]:
bm25_vclaim_results.head()

iclaim_id  Q0                                     vclaim_id  rank  \
0  tweet-sno-999  Q0    vclaim-sno-single-mother-birth-certificate     1   
1  tweet-sno-999  Q0  vclaim-sno-sick-child-christmas-card-request     2   
2  tweet-sno-999  Q0      vclaim-sno-fairfax-county-racism-posting     3   
3  tweet-sno-999  Q0                     vclaim-sno-dmx-death-hoax     4   
4  tweet-sno-999  Q0    vclaim-sno-jennifer-aniston-homeless-story     5   

       score  method  
0  23.146533  elasic  
1  14.071083  elasic  
2  13.431669  elasic  
3  12.909258  elasic  
4  12.785095  elasic

In [204]:
bm25_title_results.head()

iclaim_id  Q0                                   vclaim_id  rank  \
0  tweet-sno-999  Q0  vclaim-sno-single-mother-birth-certificate     1   
1  tweet-sno-999  Q0         vclaim-sno-mother-and-child-reunion     2   
2  tweet-sno-999  Q0      vclaim-sno-obama-spying-trump-campaign     3   
3  tweet-sno-999  Q0     vclaim-sno-being-single-is-a-disability     4   
4  tweet-sno-999  Q0                      vclaim-sno-ice-baiting     5   

       score  method  
0  26.398872  elasic  
1  13.891885  elasic  
2   9.583378  elasic  
3   9.405990  elasic  
4   9.077179  elasic

In [119]:
sbert_results.head()

iclaim_id  Q0                                          vclaim_id  rank  \
0  tweet-sno-999  Q0         vclaim-sno-single-mother-birth-certificate     1   
1  tweet-sno-999  Q0    vclaim-sno-tennessee-bill-children-illegitimate     2   
2  tweet-sno-999  Q0   vclaim-sno-birth-certificates-financial-accounts     3   
3  tweet-sno-999  Q0       vclaim-sno-texas-birth-certificates-illegals     4   
4  tweet-sno-999  Q0  vclaim-sno-barron-trump-benefit-birthright-cit...     5   

      score method  
0  0.575202  sbert  
1  0.474206  sbert  
2  0.471895  sbert  
3  0.464378  sbert  
4  0.445186  sbert

In [205]:
i = 0
qid = 1
index = 0
start_index = 0
rank_svm_info = []
main_results = bm25_all_results
while index < len(main_results):
    if i < N:
        i = i + 1
        bm25_title_features = get_rr_score(bm25_title_results, 
                                           main_results.loc[index, 'iclaim_id'], 
                                           main_results.loc[index, 'vclaim_id'], 
                                           start_index)
        
        bm25_vclaim_features = get_rr_score(bm25_vclaim_results, 
                                            main_results.loc[index, 'iclaim_id'], 
                                            main_results.loc[index, 'vclaim_id'], 
                                            start_index)
        
        sbert_features = get_rr_score(sbert_results, 
                                      main_results.loc[index, 'iclaim_id'], 
                                      main_results.loc[index, 'vclaim_id'], 
                                      start_index)
        
#         bm25_all_features = get_rr_score(bm25_all_results, 
#                                       main_results.loc[index, 'iclaim_id'], 
#                                       main_results.loc[index, 'vclaim_id'], 
#                                       start_index)
        
        
        rank_svm_info.append(str(i) + ' qid:' + str(qid) 
                             + ' 1:' + str(main_results.loc[index, 'score'])
                             + ' 2:' + str(1/main_results.loc[index, 'rank'])
                             + ' 3:' + str(bm25_title_features[0]) 
                             + ' 4:' + str(bm25_title_features[1]) 
                             + ' 5:' + str(bm25_vclaim_features[0]) 
                             + ' 6:' + str(bm25_vclaim_features[1]) 
                             + ' 7:' + str(sbert_features[0]) 
                             + ' 8:' + str(sbert_features[1]) 
                            )

#         rank_svm_info.append(str(i) + ' qid:' + str(qid) 
#                          + ' 1:' + str(main_results.loc[index, 'score'])
#                          + ' 2:' + str(1/main_results.loc[index, 'rank'])
#                          + ' 3:' + str(bm25_title_features[0]) 
#                          + ' 4:' + str(bm25_title_features[1]) 
#                          + ' 5:' + str(bm25_vclaim_features[0]) 
#                          + ' 6:' + str(bm25_vclaim_features[1]) 
#                          + ' 7:' + str(bm25_all_features[0]) 
#                          + ' 8:' + str(bm25_all_features[1]) 
#                         )
        index = index + 1

    else:
        print(qid)
        qid = qid + 1
        i = 0
        index = index - N + cnt_vclaims
        start_index = index


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [206]:
print(rank_svm_info)

['1 qid:1 1:24.39000186062631 2:1.0 3:26.398872015567704 4:1.0 5:23.14653325630984 6:1.0 7:0.5752015113830566 8:1.0', '2 qid:1 1:18.51070934698121 2:0.5 3:0.0 4:0.00015035333032626674 5:5.204186078367649 6:0.0035971223021582736 7:0.4332945048809051 8:0.125', '3 qid:1 1:14.801876057059255 2:0.3333333333333333 3:0.0 4:0.00021199915200339198 5:13.43166919304542 6:0.3333333333333333 7:0.2222197949886322 8:0.00040749796251018743', '4 qid:1 1:14.652756782684657 2:0.25 3:6.380044265404401 4:0.012345679012345678 5:0.0 6:8.229775327133569e-05 7:0.1778967678546905 8:0.00019171779141104295', '5 qid:1 1:14.346758792837708 2:0.2 3:5.96285369033028 4:0.009615384615384616 5:5.653105626136733 6:0.006097560975609756 7:0.3588125705718994 8:0.01282051282051282', '6 qid:1 1:12.72657835278109 2:0.16666666666666666 3:5.3952720243767285 4:0.00510204081632653 5:14.071082752978256 6:0.5 7:0.3705777525901794 8:0.018867924528301886', '7 qid:1 1:12.71508311246333 2:0.14285714285714285 3:7.485850386815596 4:0.0384

In [209]:
with open('subtask2a/rank_top20', 'w') as f:
    for item in rank_svm_info:
        f.write("%s\n" % item)

In [210]:
#TRAIN ON RANK SVM
#svm_rank_learn -c 38 subtask2a/rank_top20 subtask2a/model_top20_linear
#test: svm_rank_classify subtask2a/rank_top20 subtask2a/model_top20_linear subtask2a/predictions_top20_linear


#load rank svm results
#rank_svm_results = open("predictions", "r")
rank_svm_results = open("subtask2a/predictions_top20_linear", "r")
predictions = rank_svm_results.read().split('\n')
print(predictions)
close(rank_svm_results)

['-302.75107286', '-228.36929025', '-182.36973294', '-180.00234446', '-176.05057330', '-156.22425157', '-155.85482953', '-151.92872058', '-149.30084720', '-143.49120787', '-142.25839089', '-140.41693659', '-135.29248986', '-132.11078042', '-131.10845989', '-129.29562818', '-128.87285062', '-127.24653817', '-126.13074684', '-124.97043532', '-374.51764011', '-271.04529557', '-197.28505670', '-187.34052501', '-187.09324815', '-183.68931403', '-163.30523164', '-163.26761271', '-158.46853270', '-154.13058706', '-150.43944699', '-140.78695796', '-140.75940061', '-139.59657390', '-139.44583999', '-137.28731134', '-136.89527195', '-136.20463810', '-135.97901238', '-135.41336905', '-347.77168832', '-225.52778735', '-195.05613403', '-192.36042176', '-183.29449566', '-180.52054678', '-180.43890257', '-180.31304553', '-180.25970603', '-176.48113500', '-173.15548229', '-170.80761003', '-168.64784103', '-166.62340793', '-166.57263357', '-166.56145739', '-166.55539499', '-156.45589080', '-154.9683125

NameError: name 'close' is not defined

In [211]:
predictions = np.array(predictions[:-1]).reshape(cnt_iclaims, N)

In [212]:
print(predictions)

[['-302.75107286' '-228.36929025' '-182.36973294' ... '-127.24653817'
  '-126.13074684' '-124.97043532']
 ['-374.51764011' '-271.04529557' '-197.28505670' ... '-136.20463810'
  '-135.97901238' '-135.41336905']
 ['-347.77168832' '-225.52778735' '-195.05613403' ... '-156.45589080'
  '-154.96831252' '-154.92644847']
 ...
 ['-184.50535462' '-182.10773794' '-181.31215096' ... '-149.72018634'
  '-148.07157480' '-148.05859547']
 ['-231.97969899' '-220.37284037' '-219.02440419' ... '-138.21953142'
  '-136.67106789' '-132.25261904']
 ['-172.81376086' '-136.08487029' '-124.18726134' ... '-96.47985190'
  '-95.71029135' '-92.84144430']]


In [213]:
iclaim_index = 0
index = 0
all_scores = {}
while index < len(bm25_vclaim_results):
    
    iclaim_id = bm25_vclaim_results.loc[index, 'iclaim_id']
    scores = {}
    i = 0
    
    while i < N:
        scores[bm25_vclaim_results.loc[index, 'vclaim_id']] = -float(predictions[iclaim_index][i]) 
        # ranksvm scores sort queries in ascending order
        # evaluation requires the first queries to have the highes score
        i = i + 1
        index = index + 1
    
    scores = sorted(list(scores.items()), key=lambda kv: kv[1], reverse=True)
    print(scores)
    all_scores[iclaim_id] = scores
    iclaim_index = iclaim_index + 1
    index = index - N + cnt_vclaims

[('vclaim-sno-single-mother-birth-certificate', 302.75107286), ('vclaim-sno-sick-child-christmas-card-request', 228.36929025), ('vclaim-sno-fairfax-county-racism-posting', 182.36973294), ('vclaim-sno-dmx-death-hoax', 180.00234446), ('vclaim-sno-jennifer-aniston-homeless-story', 176.0505733), ('vclaim-sno-stephen-moore-child-labor', 156.22425157), ('vclaim-sno-youngest-mother', 155.85482953), ('vclaim-sno-le-a', 151.92872058), ('vclaim-sno-one-eyed-mother', 149.3008472), ('vclaim-sno-debbie-shwartz', 143.49120787), ('vclaim-sno-muslims-declare-december-24th-islamic-holiday', 142.25839089), ('vclaim-sno-trump-president-drafted', 140.41693659), ('vclaim-sno-anthony-southern', 135.29248986), ('vclaim-sno-kamala-harris-homeless-mother', 132.11078042), ('vclaim-sno-quake-mother', 131.10845989), ('vclaim-sno-covid-nyc-tokyo', 129.29562818), ('vclaim-sno-lindsey-graham-statement-presidential-race-fake', 128.87285062), ('vclaim-sno-black-panthers-wic-food-program', 127.24653817), ('vclaim-sno-m

In [214]:
print(all_scores)

{'tweet-sno-999': [('vclaim-sno-single-mother-birth-certificate', 302.75107286), ('vclaim-sno-sick-child-christmas-card-request', 228.36929025), ('vclaim-sno-fairfax-county-racism-posting', 182.36973294), ('vclaim-sno-dmx-death-hoax', 180.00234446), ('vclaim-sno-jennifer-aniston-homeless-story', 176.0505733), ('vclaim-sno-stephen-moore-child-labor', 156.22425157), ('vclaim-sno-youngest-mother', 155.85482953), ('vclaim-sno-le-a', 151.92872058), ('vclaim-sno-one-eyed-mother', 149.3008472), ('vclaim-sno-debbie-shwartz', 143.49120787), ('vclaim-sno-muslims-declare-december-24th-islamic-holiday', 142.25839089), ('vclaim-sno-trump-president-drafted', 140.41693659), ('vclaim-sno-anthony-southern', 135.29248986), ('vclaim-sno-kamala-harris-homeless-mother', 132.11078042), ('vclaim-sno-quake-mother', 131.10845989), ('vclaim-sno-covid-nyc-tokyo', 129.29562818), ('vclaim-sno-lindsey-graham-statement-presidential-race-fake', 128.87285062), ('vclaim-sno-black-panthers-wic-food-program', 127.2465381

In [215]:
def format_scores(scores):
    output_string = ''
    i = 1
    for iclaim_id in scores:
        i = i + 1
        for i, (vclaim_id, score) in enumerate(scores[iclaim_id]):
            output_string += f"{iclaim_id}\tQ0\t{vclaim_id}\t{i + 1}\t{score}\trank_svm\n"
    return output_string

In [216]:
formatted_scores = format_scores(all_scores)
rank_svm_file = 'rank_top20_linear.tsv'
with open(rank_svm_file, 'w') as f:
    f.write(formatted_scores)

In [217]:
from scorer.main import evaluate
import logging

dev_file_path = 'Project/Fact-checked-Claim-Detection/data/subtask-2a--english/qrels-dev.tsv'

maps, mrr, precisions = evaluate(dev_file_path, rank_svm_file)
logging.info(f'All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. {maps}')
logging.info(f'MRR score {mrr}')
logging.info(f'All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. {precisions}')

INFO : All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.6381909547738693, 0.6825795644891122, 0.6903685092127303, 0.6952939299672968, 0.6988790104367995, 0.6988790104367995, 0.6988790104367995]
INFO : MRR score 0.6987115062491946
INFO : All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.6381909547738693, 0.2445561139028475, 0.15376884422110548, 0.08040201005025124, 0.04296482412060301, 0.017185929648241206, 0.0008592964824120606]
